# Defining Your Own Python Function, Part #2

## Table of Contents 
* [Variable-Length Argument Lists](#variable-length_argument_lists)
    * [Argument Tuple Packing](#argument_tuple_packing)
    * [Argument Tuple Unpacking](#argument_tuple_unpacking)
    * [Argument Dictionary Packing](#argument_dictionary_packing)
    * [Argument Dictionary Unpacking](#argument_dictionary_unpacking)
    * [Putting It All Together](#putting_it_all_together)
    * [Multiple Unpackings in a Python Function Call](#multiple_unpackings_in_a_python_function_call)
* [Keyword-Only Arguments](#keyword-only_arguments)
* [Positional-Only Arguments](#positional-only_arguments)
* [Docstrings](#docstrings)
* [Python Function Annotations](#python_function_annotations)
* [Conclusion](#conclusion)

---

Now that you've learned the basics of functions, let's explore how we can make it more flexible and advanced.

<a class="anchor" id="variable-length_argument_lists"></a>
## Variable-Length Argument Lists

In some cases, when you’re defining a function, you may not know beforehand how many arguments you’ll want it to take. Suppose, for example, that you want to write a Python function that computes the average of several values. You could start with something like this:

In [ ]:
def avg(a, b, c):
    return (a + b + c) / 3

All is well if you want to average three values:

In [ ]:
avg(1, 2, 3)

However, as you’ve already seen, when positional arguments are used, the number of arguments passed must agree with the number of parameters declared. Clearly then, all isn’t well with this implementation of `avg()` for any number of values other than three:

In [ ]:
avg(1, 2, 3, 4)

You could try to define `avg()` with optional parameters:
```python
def avg(a, b=0, c=0, d=0, e=0):
    .
    .
    .
```

This allows for a variable number of arguments to be specified. The following calls are at least syntactically correct:

```python
avg(1)
avg(1, 2)
avg(1, 2, 3)
avg(1, 2, 3, 4)
avg(1, 2, 3, 4, 5)
```

But this approach still suffers from a couple of problems. For starters, it still only allows up to five arguments, not an arbitrary number. Worse yet, there’s no way to distinguish between the arguments that were specified and those that were allowed to default. The function has no way to know how many arguments were actually passed, so it doesn’t know what to divide by:

```python
def avg(a, b=0, c=0, d=0, e=0):
    return (a + b + c + d + e) / # Divided by what???
```

Evidently, this won’t do either.

You could write `avg()` to take a single list argument:

In [ ]:
def avg(a):
    total = 0
    for v in a:
        total += v
    return total / len(a)

In [ ]:
avg([1, 2, 3])

In [ ]:
avg([1, 2, 3, 4, 5])

At least this works. It allows an arbitrary number of values and produces a correct result. As an added bonus, it works when the argument is a tuple as well:



In [ ]:
t = (1, 2, 3, 4, 5)
avg(t)

The drawback is that the added step of having to group the values into a list or tuple is probably not something the user of the function would expect, and it isn’t very elegant. Whenever you find Python code that looks inelegant, there’s probably a better option.

In this case, indeed there is! Python provides a way to pass a function a variable number of arguments with argument tuple packing and unpacking using the asterisk (`*`) operator.

<a class="anchor" id="argument_tuple_packing"></a>
### Argument Tuple Packing

When a parameter name in a Python function definition is preceded by an asterisk (`*`), it indicates **argument tuple packing**. Any corresponding arguments in the function call are packed into a tuple that the function can refer to by the given parameter name. Here’s an example:

In [ ]:
def f(*args):
    print(args)
    print(type(args), len(args))
    for x in args:
        print(x)

In [ ]:
f(1, 2, 3)

In [ ]:
f('foo', 'bar', 'baz', 'qux', 'quux')

In the definition of `f()`, the parameter specification `*args` indicates tuple packing. In each call to `f()`, the arguments are packed into a tuple that the function can refer to by the name `args`. Any name can be used, but `args` is so commonly chosen that it’s practically a standard.

Using tuple packing, you can clean up `avg()` like this:

In [ ]:
def avg(*args):
    total = 0
    for i in args:
        total += i
    return total / len(args)

In [ ]:
avg(1, 2, 3)

In [ ]:
avg(1, 2, 3, 4, 5)

Better still, you can tidy it up even further by replacing the `for` loop with the built-in Python function `sum()`, which sums the numeric values in any iterable:

In [ ]:
def avg(*args):
    return sum(args) / len(args)

In [ ]:
avg(1, 2, 3)

In [ ]:
avg(1, 2, 3, 4, 5)

Now, `avg()` is concisely written and works as intended.

Still, depending on how this code will be used, there may still be work to do. As written, `avg()` will produce a `TypeError` exception if any arguments are non-numeric:

In [ ]:
avg(1, 'foo', 3)

To be as robust as possible, you should add code to check that the arguments are of the proper type. Later in this tutorial series, you’ll learn how to catch exceptions like `TypeError` and handle them appropriately. 

<a class="anchor" id="argument_tuple_unpacking"></a>
### Argument Tuple Unpacking

An analogous operation is available on the other side of the equation in a Python function call. When an argument in a function call is preceded by an asterisk (`*`), it indicates that the argument is a tuple that should be **unpacked** and passed to the function as separate values:

In [ ]:
def f(x, y, z):
    print(f'x = {x}')
    print(f'y = {y}')
    print(f'z = {z}')

In [ ]:
f(1, 2, 3)

In [ ]:
t = ('foo', 'bar', 'baz')

In [ ]:
f(*t)

In this example, `*t` in the function call indicates that `t` is a tuple that should be unpacked. The unpacked values `'foo'`, `'bar'`, and `'baz'` are assigned to the parameters `x`, `y`, and `z`, respectively.

Although this type of unpacking is called **tuple** unpacking, it doesn’t only work with tuples. The asterisk (`*`) operator can be applied to any iterable in a Python function call. For example, a list or set can be unpacked as well:

In [ ]:
a = ['foo', 'bar', 'baz']
type(a)

In [ ]:
f(*a)

In [ ]:
s = {1, 2, 3}
type(s)

In [ ]:
f(*s)

You can even use tuple packing and unpacking at the same time:

In [ ]:
def f(*args):
    print(type(args), args)

In [ ]:
a = ['foo', 'bar', 'baz', 'qux']
f(*a)

Here, `f(*a)` indicates that list a should be unpacked and the items passed to `f()` as individual values. The parameter specification `*args` causes the values to be packed back up into the tuple args.

<a class="anchor" id="argument_dictionary_packing"></a>
### Argument Dictionary Packing

Python has a similar operator, the double asterisk (`**`), which can be used with Python function parameters and arguments to specify **dictionary packing and unpacking**. Preceding a parameter in a Python function definition by a double asterisk (`**`) indicates that the corresponding arguments, which are expected to be `key=value` pairs, should be packed into a dictionary:

In [ ]:
def f(**kwargs):
    print(kwargs)
    print(type(kwargs))
    for key, val in kwargs.items():
        print(key, '->', val)

In [ ]:
f(foo=1, bar=2, baz=3)

In this case, the arguments `foo=1`, `bar=2`, and `baz=3` are packed into a dictionary that the function can reference by the name `kwargs`. Again, any name can be used, but the peculiar `kwargs` (which is short for **keyword args**) is nearly standard. You don’t have to adhere to it, but if you do, then anyone familiar with Python coding conventions will know straightaway what you mean.

<a class="anchor" id="argument_dictionary_unpacking"></a>
### Argument Dictionary Unpacking

**Argument dictionary unpacking** is analogous to argument tuple unpacking. When the double asterisk (`**`) precedes an argument in a Python function call, it specifies that the argument is a dictionary that should be unpacked, with the resulting items passed to the function as keyword arguments:

In [ ]:
def f(a, b, c):
    print(F'a = {a}')
    print(F'b = {b}')
    print(F'c = {c}')

In [ ]:
d = {'a': 'foo', 'b': 25, 'c': 'qux'}

In [ ]:
f(**d)

The items in the dictionary `d` are unpacked and passed to `f()` as keyword arguments. So, `f(**d)` is equivalent to `f(a='foo', b=25, c='qux')`:

In [ ]:
f(a='foo', b=25, c='qux')

In fact, check this out:

In [ ]:
f(**dict(a='foo', b=25, c='qux'))

Here, `dict(a='foo', b=25, c='qux')` creates a dictionary from the specified key/value pairs. Then, the double asterisk operator (`**`) unpacks it and passes the keywords to `f()`.

<a class="anchor" id="putting_it_all_together"></a>
### Putting It All Together

Think of `*args` as a variable-length positional argument list, and `**kwargs` as a variable-length keyword argument list.

All three—standard positional parameters, `*args`, and `**kwargs`—can be used in one Python function definition. If so, then they should be specified in that order:

In [ ]:
def f(a, b, *args, **kwargs):
    print(F'a = {a}')
    print(F'b = {b}')
    print(F'args = {args}')
    print(F'kwargs = {kwargs}')

In [ ]:
f(1, 2, 'foo', 'bar', 'baz', 'qux', x=100, y=200, z=300)

This provides just about as much flexibility as you could ever need in a function interface!

<a class="anchor" id="multiple_unpackings_in_a_python_function_call"></a>
### Multiple Unpackings in a Python Function Call

Python `version 3.5` introduced support for additional unpacking generalizations, as outlined in [PEP 448](https://www.python.org/dev/peps/pep-0448). One thing these enhancements allow is **multiple unpackings** in a single Python function call:

In [ ]:
def f(*args):
    for i in args:
        print(i)

In [ ]:
a = [1, 2, 3]
t = (4, 5, 6)
s = {7, 8, 9}

In [ ]:
f(*a, *t, *s)

You can specify multiple dictionary unpackings in a Python function call as well:

In [ ]:
def f(**kwargs):
    for k, v in kwargs.items():
        print(k, '->', v)

In [ ]:
d1 = {'a': 1, 'b': 2}
d2 = {'x': 3, 'y': 4}

In [ ]:
f(**d1, **d2)

> **Note:** This enhancement is available only in Python `version 3.5` or later. If you try this in an earlier version, then you’ll get a `SyntaxError` exception.

By the way, the unpacking operators `*` and `**` don’t apply only to variables, as in the examples above. You can also use them with literals that are iterable:

In [ ]:
def f(*args):
    for i in args:
        print(i)

In [ ]:
f(*[1, 2, 3], *[4, 5, 6])

In [ ]:
def f(**kwargs):
    for k, v in kwargs.items():
        print(k, '->', v)

In [ ]:
f(**{'a': 1, 'b': 2}, **{'x': 3, 'y': 4})

Here, the literal lists `[1, 2, 3]` and `[4, 5, 6]` are specified for tuple unpacking, and the literal dictionaries `{'a': 1, 'b': 2}` and `{'x': 3, 'y': 4}` are specified for dictionary unpacking.

<a class="anchor" id="keyword-only_arguments"></a>
## Keyword-Only Arguments

A Python function in `version 3.x` can be defined so that it takes **keyword-only arguments**. These are function arguments that must be specified by keyword. Let’s explore a situation where this might be beneficial.

Suppose you want to write a Python function that takes a variable number of string arguments, concatenates them together separated by a dot (`"."`), and prints them to the console. Something like this will do to start:

In [ ]:
def concat(*args):
    print(f'-> {".".join(args)}')

In [ ]:
concat('a', 'b', 'c', 'd')

In [ ]:
concat('foo', 'bar', 'baz', 'qux')

As it stands, the output prefix is hard-coded to the string `'-> '`. What if you want to modify the function to accept this as an argument as well, so the user can specify something else? This is one possibility:

In [ ]:
def concat(*args):
    print(f'-> {".".join(args)}')

In [ ]:
concat('a', 'b', 'c')

In [ ]:
concat('foo', 'bar', 'baz', 'qux')

As it stands, the output prefix is hard-coded to the string `'-> '`. What if you want to modify the function to accept this as an argument as well, so the user can specify something else? This is one possibility:

In [ ]:
def concat(prefix, *args):
    print(f'{prefix}{".".join(args)}')

In [ ]:
concat('//', 'a', 'b', 'c')

In [ ]:
concat('... ', 'foo', 'bar', 'baz', 'qux')

This works as advertised, but there are a couple of undesirable things about this solution:

- The `prefix` string is lumped together with the strings to be concatenated. Just from looking at the function call, it isn’t clear that the first argument is treated differently from the rest. To know that, you’d have to go back and look at the function definition.

- `prefix` isn’t optional. It always has to be included, and there’s no way to assume a default value.

You might think you could overcome the second issue by specifying a parameter with a default value, like this, perhaps:

In [ ]:
def concat(prefix='-> ', *args):
    print(f'{prefix}{".".join(args)}')

Unfortunately, this doesn’t work quite right. `prefix` is a **positional parameter**, so the interpreter assumes that the first argument specified in the function call is the intended output prefix. This means there isn’t any way to omit it and obtain the default value:

In [ ]:
concat('a', 'b', 'c')

What if you try to specify `prefix` as a keyword argument? Well, you can’t specify it first:

In [ ]:
concat(prefix='//', 'a', 'b', 'c')

As you’ve seen previously, when both types of arguments are given, all positional arguments must come before any keyword arguments.

However, you can’t specify it last either:

In [ ]:
concat('a', 'b', 'c', prefix='... ')

Again, `prefix` is a positional parameter, so it’s assigned the first argument specified in the call (which is `'a'` in this case). Then, when it’s specified again as a keyword argument at the end, Python thinks it’s been assigned twice.

**Keyword-only parameters** help solve this dilemma. In the function definition, specify `*args` to indicate a variable number of positional arguments, and then specify `prefix` after that:

In [ ]:
def concat(*args, prefix='-> '):
    print(f'{prefix}{".".join(args)}')

In that case, `prefix` becomes a keyword-only parameter. Its value will never be filled by a positional argument. It can only be specified by a named keyword argument:

In [ ]:
concat('a', 'b', 'c', prefix='... ')

Note that this is only possible in Python 3. In versions 2.x of Python, specifying additional parameters after the `*args` variable arguments parameter raises an error.

Keyword-only arguments allow a Python function to take a variable number of arguments, followed by one or more additional **options** as keyword arguments. If you wanted to modify `concat()` so that the separator character can optionally be specified as well, then you could add an additional keyword-only argument:

In [ ]:
def concat(*args, prefix='-> ', sep='.'):
    print(f'{prefix}{sep.join(args)}')

In [ ]:
concat('a', 'b', 'c')

In [ ]:
concat('a', 'b', 'c', prefix='//')

In [ ]:
concat('a', 'b', 'c', prefix='//', sep='-')

If a keyword-only parameter is given a default value in the function definition (as it is in the example above), and the keyword is omitted when the function is called, then the default value is supplied:

In [ ]:
concat('a', 'b', 'c')

If, on the other hand, the parameter isn’t given a default value, then it becomes required, and failure to specify it results in an error:

In [ ]:
def concat(*args, prefix):
    print(f'{prefix}{".".join(args)}')

In [ ]:
concat('a', 'b', 'c', prefix='... ')

In [ ]:
concat('a', 'b', 'c')

What if you want to define a Python function that takes a keyword-only argument but doesn’t take a variable number of positional arguments? For example, the following function performs the specified operation on two numerical arguments:

In [ ]:
def oper(x, y, op='+'):
    if op == '+':
        return x + y
    elif op == '-':
        return x - y
    elif op == '/':
        return x / y
    else:
        return None

In [ ]:
oper(3, 4)

In [ ]:
oper(3, 4, '+')

In [ ]:
oper(3, 4, '/')

If you wanted to make op a keyword-only parameter, then you could add an extraneous dummy variable argument parameter and just ignore it:

In [ ]:
def oper(x, y, *ignore, op='+'):
    if op == '+':
        return x + y
    elif op == '-':
        return x - y
    elif op == '/':
        return x / y
    else:
        return None

In [ ]:
oper(3, 4, op='+')

In [ ]:
oper(3, 4, op='/')

The problem with this solution is that `*ignore` absorbs any extraneous positional arguments that might happen to be included:

In [ ]:
oper(3, 4, "I don't belong here")

In [ ]:
oper(3, 4, "I don't belong here", op='/')

In this example, the extra argument shouldn’t be there (as the argument itself announces). Instead of quietly succeeding, it should really result in an error. The fact that it doesn’t is untidy at best. At worst, it may cause a result that appears misleading:

In [ ]:
oper(3, 4, '/')

To remedy this, version 3 allows a **variable argument parameter** in a Python function definition to be just a bare asterisk (`*`), with the name omitted:

In [ ]:
def oper(x, y, *, op='+'):
    if op == '+':
        return x + y
    elif op == '-':
        return x - y
    elif op == '/':
        return x / y
    else:
        return None

In [ ]:
oper(3, 4, op='+')

In [ ]:
oper(3, 4, op='/')

In [ ]:
oper(3, 4, "I don't belong here")

In [ ]:
oper(3, 4, '+')

The **bare variable argument parameter** `*` indicates that there aren’t any more positional parameters. This behavior generates appropriate error messages if extra ones are specified. It allows keyword-only parameters to follow.

<a class="anchor" id="positional-only_arguments"></a>
## Positional-Only Arguments

As of Python 3.8, function parameters can also be declared **positional-only**, meaning the corresponding arguments must be supplied positionally and can’t be specified by keyword.

To designate some parameters as positional-only, you specify a bare slash (`/`) in the parameter list of a function definition. Any parameters to the left of the slash (`/`) must be specified positionally. For example, in the following function definition, `x` and `y` are positional-only parameters, but `z` may be specified by keyword:

In [ ]:
# This is Python 3.8
def f(x, y, /, z):
    print(f'x: {x}')
    print(f'y: {y}')
    print(f'z: {z}')

This means that the following calls are valid:

In [ ]:
f(1, 2, 3)

In [ ]:
f(1, 2, z=3)

The following call to `f()`, however, is not valid:

In [ ]:
f(x=1, y=2, z=3)

The positional-only and keyword-only designators may both be used in the same function definition:

In [ ]:
# This is Python 3.8
def f(x, y, /, z, w, *, a, b):
    print(x, y, z, w, a, b)

In [ ]:
f(1, 2, z=3, w=4, a=5, b=6)

In [ ]:
f(1, 2, 3, w=4, a=5, b=6)

In this example:

- `x` **and** `y` are positional-only.
- `a` **and** `b` are keyword-only.
- `z` **and** `w` may be specified positionally or by keyword.

<a class="anchor" id="docstrings"></a>
## Docstrings

When the first statement in the body of a Python function is a string literal, it’s known as the function’s **docstring**. A docstring is used to supply documentation for a function. It can contain the function’s purpose, what arguments it takes, information about return values, or any other information you think would be useful.

The following is an example of a function definition with a docstring:

In [ ]:
def avg(*args):
    """Returns the average of a list of numeric values."""
    return sum(args) / len(args)

Technically, docstrings can use any of Python’s quoting mechanisms, but the recommended convention is to **triple-quote** using double-quote characters (`"""`), as shown above. If the docstring fits on one line, then the closing quotes should be on the same line as the opening quotes.

**Multi-line docstrings** are used for lengthier documentation. A multi-line docstring should consist of a summary line, followed by a blank line, followed by a more detailed description. The closing quotes should be on a line by themselves:

In [ ]:
def foo(bar=0, baz=1):
    """Perform a foo transformation.

    Keyword arguments:
    bar -- magnitude along the bar axis (default=0)
    baz -- magnitude along the baz axis (default=1)
    """
    # <function_body>

Docstring formatting and semantic conventions are detailed in [PEP 257](https://www.python.org/dev/peps/pep-0257).

When a docstring is defined, the Python interpreter assigns it to a special attribute of the function called `__doc__`. This attribute is one of a set of specialized identifiers in Python that are sometimes called **magic attributes** or **magic methods** because they provide special language functionality.

> **Note:** These attributes are also referred to by the colorful nickname dunder attributes and dunder methods. The word **dunder** combines the **d** from double and **under** from the underscore character (`_`). You’ll encounter many more dunder attributes and methods in future tutorials in this series.

You can access a function’s docstring with the expression `<function_name>.__doc__`. The docstrings for the above examples can be displayed as follows:

In [ ]:
print(avg.__doc__)

In [ ]:
print(foo.__doc__)

In the interactive Python interpreter, you can type `help(<function_name>)` to display the docstring for `<function_name>`:

In [ ]:
help(avg)

In [ ]:
help(foo)

It’s considered good coding practice to specify a docstring for each Python function you define. Documenting Python Code will be covered later.

<a class="anchor" id="python_function_annotations"></a>
## Python Function Annotations

As of version 3.0, Python provides an additional feature for documenting a function called a **function annotation**. Annotations provide a way to attach metadata to a function’s parameters and return value.

In [ ]:
def f(a: float, b: float = 8) -> int:
    """Add two number.

    :param a: First input
    :param b: Second input
    :return: Sum of two numbers.
    """
    
    return a + b

To add an annotation to a Python function parameter, insert a colon (`:`) followed by any expression after the parameter name in the function definition. To add an annotation to the return value, add the characters `->` and any expression between the closing parenthesis of the parameter list and the colon that terminates the function header. Here’s an example:

```python
def f(a: '<a>', b: '<b>') -> '<ret_value>':
    pass
```

The annotation for parameter `a` is the string `'<a>'`, for `b` the string `'<b>'`, and for the function return value the string `'<ret_value>'`.

The Python interpreter creates a dictionary from the annotations and assigns them to another special dunder attribute of the function called `__annotations__`. The annotations for the Python function `f()` shown above can be displayed as follows:

In [ ]:
f.__annotations__

The keys for the parameters are the parameter names. The key for the return value is the string `'return'`:

In [ ]:
f.__annotations__['a']

In [ ]:
f.__annotations__['b']

In [ ]:
f.__annotations__['return']

Note that annotations aren’t restricted to string values. They can be any expression or object. For example, you might annotate with type objects:

In [ ]:
def f(a: int, b: str) -> float:
    print(a, b)
    return(3.5)

In [ ]:
f(1, 'foo')

In [ ]:
f.__annotations__

An annotation can even be a composite object like a list or a dictionary, so it’s possible to attach multiple items of metadata to the parameters and return value:

In [ ]:
def area(
    r: {
           'desc': 'radius of circle',
           'type': float
       }) -> \
       {
           'desc': 'area of circle',
           'type': float
       }:
    return 3.14159 * (r ** 2)

In [ ]:
area(2.5)

In [ ]:
area.__annotations__

In [ ]:
area.__annotations__['r']['desc']

In [ ]:
area.__annotations__['return']['type']

In the example above, an annotation is attached to the parameter `r` and to the return value. Each annotation is a dictionary containing a string description and a type object.

If you want to assign a default value to a parameter that has an annotation, then the default value goes after the annotation:

In [ ]:
def f(a: int = 12, b: str = 'baz') -> float:
    print(a, b)
    return(3.5)

In [ ]:
f.__annotations__

In [ ]:
f()

What do annotations do? Frankly, they don’t do much of anything. They’re just kind of there. Let’s look at one of the examples from above again, but with a few minor modifications:

In [ ]:
def f(a: int, b: str) -> float:
    print(a, b)
    return 1, 2, 3

In [ ]:
f('foo', 2.5)

What’s going on here? The annotations for `f()` indicate that the first argument is `int`, the second argument `str`, and the return value `float`. But the subsequent call to `f()` breaks all the rules! The arguments are `str` and `float`, respectively, and the return value is a `tuple`. Yet the interpreter lets it all slide with no complaint at all.

Annotations don’t impose any **semantic restrictions** on the code whatsoever. They’re simply bits of metadata attached to the Python function parameters and return value. Python dutifully stashes them in a dictionary, assigns the dictionary to the function’s `__annotations__` dunder attribute, and that’s it. Annotations are completely optional and don’t have any impact on Python function execution at all.

For starters, annotations make good **documentation**. You can specify the same information in the docstring, of course, but placing it directly in the function definition adds clarity. The types of the arguments and the return value are obvious on sight for a function header like this:

```python
def f(a: int, b: str) -> float:
```

Granted, the interpreter doesn’t enforce adherence to the types specified, but at least they’re clear to someone reading the function definition.

There’s another benefit to using annotations as well. The standardized format in which annotation information is stored in the `__annotations__` attribute lends itself to the parsing of function signatures by automated tools.

Python function annotations are nothing more than dictionaries of metadata. It just happens that you can create them with convenient syntax that’s supported by the interpreter. They’re whatever you choose to make of them.

<a class="anchor" id="conclusion"></a>
## Conclusion 
As applications grow larger, it becomes increasingly important to modularize code by breaking it up into smaller functions of manageable size. You now hopefully have all the tools you need to do this.

You’ve learned:

- How to create variable length arguments for functions
- Keyword only and positional only arguments
- How to add documentation to functions with **docstrings** and **annotations**

Next up in this series, you’ll explore how Python avoids conflict between identifiers in different areas of code. As you’ve already seen, each function in Python has its own namespace, distinct from those of other functions. In the next tutorial, you’ll learn how namespaces are implemented in Python and how they define variable **scope**.